## UAV fleet Simulation 

This is for testing propouses

In [1]:
import numpy as np

# Custom modules
from helpers import local2global, plot_3d_interactive, kill_processes
from simulators import Simulator, Gazebo, QGC

from plan import Plan, State
from env import Enviroment

kill_processes()

## Create Plans

In [2]:
offsets = [(0, 0, 0, 0)]  # east, north, up, heading
n_vehicles = len(offsets)
local_paths = [Plan.create_square_path(side_len=3, alt=5)]
plans = [Plan.basic(wps=path, alt=5, wp_margin=0.5) for path in local_paths]

## Visualization Parameters

In [3]:
homes = np.array([offset[:3] for offset in offsets])
global_paths = [
    local2global(np.array(path), home, pairwise=True)
    for path, home in zip(local_paths, homes)
]
colors = ["Blue"]  # must agree with n_vehicles
waypoints = {
    f"waypoint_{i+1}": {"pos": global_paths[i], "color": colors[i]}
    for i in range(n_vehicles)
}
plot_3d_interactive(
    waypoints, title="Simulation Markers", expand=[0.2, 0.2, 0.6], ground=-0.05
)

## Choose Simulator

In [4]:
# simulator = Simulator(name="NONE",offsets=offsets,plans=plans)

# simulator=QGC(offsets=offsets,
#               plans=plans,
#               origin=(-35.3633245,149.1652241,0,0))

simulator = Gazebo(
    offsets=offsets,
    plans=plans,
    world_path="simulators/gazebo/worlds/runway.world",
    vehicle_models=n_vehicles * ["drone"],
    markers=waypoints,
)

Launch Simulator

In [5]:
uavs = simulator.launch()
env = Enviroment(uavs)

Vehicle 1 launched 🚀


## Execute Plan

In [6]:
while env.vehs:
    for uav in env.vehs.copy():
        if uav.plan.state == State.DONE:
            env.remove(uav)
        else:
            uav.act()
kill_processes()

Vehicle 1: ▶️ Starting Step: Check disarmed
Vehicle 1: ✅ Step: Check disarmed is done
Vehicle 1: ▶️ Starting Step: Check EKF
Vehicle 1: ✅ Step: Check EKF is done
Vehicle 1: ▶️ Starting Step: Check GPS
Vehicle 1: ✅ Step: Check GPS is done
Vehicle 1: ▶️ Starting Step: Check system
Vehicle 1: ✅ Step: Check system is done
Vehicle 1: ▶️ Starting Step: Switch to GUIDED
Vehicle 1: ✅ Step: Switch to GUIDED is done
Vehicle 1: ▶️ Starting Step: arm
Vehicle 1: ✅ Step: arm is done
Vehicle 1: ▶️ Starting Step: takeoff
Vehicle 1:📍 Distance to target: 5.01 m
Vehicle 1:📍 Distance to target: 5.01 m
Vehicle 1:📍 Distance to target: 5.01 m
Vehicle 1:📍 Distance to target: 5.01 m
Vehicle 1:📍 Distance to target: 5.01 m
Vehicle 1:📍 Distance to target: 5.01 m
Vehicle 1:📍 Distance to target: 5.01 m
Vehicle 1:📍 Distance to target: 5.01 m
Vehicle 1:📍 Distance to target: 5.01 m
Vehicle 1:📍 Distance to target: 5.01 m
Vehicle 1:📍 Distance to target: 5.01 m
Vehicle 1:📍 Distance to target: 5.01 m
Vehicle 1:📍 Distance 

In [7]:
uavs[0].plan

✅ <Plan 'basic'>
  ✅ <Action 'PREARM'>
    ✅ <Step 'Check disarmed'>
    ✅ <Step 'Check EKF'>
    ✅ <Step 'Check GPS'>
    ✅ <Step 'Check system'>
  ✅ <Action 'MODE: GUIDED'>
    ✅ <Step 'Switch to GUIDED'>
  ✅ <Action 'ARM'>
    ✅ <Step 'arm'>
  ✅ <Action 'TAKEOFF'>
    ✅ <Step 'takeoff'>
  ✅ <Action 'FLY'>
    ✅ <Step 'go to  -> (np.int64(0), np.int64(0), np.int64(5))'>
    ✅ <Step 'go to  -> (np.int64(0), np.int64(3), np.int64(5))'>
    ✅ <Step 'go to  -> (np.int64(3), np.int64(3), np.int64(5))'>
    ✅ <Step 'go to  -> (np.int64(3), np.int64(0), np.int64(5))'>
    ✅ <Step 'go to  -> (np.int64(0), np.int64(0), np.int64(5))'>
  ✅ <Action 'LAND'>
    ✅ <Step 'land'>

In [8]:
uav.plan.reset()